In [73]:
%matplotlib inline

import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import re
import emoji
from langdetect import detect
import unicodedata
from emoji import unicode_codes

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import neighbors

In [20]:
import datetime
import pandas as pd
from sqlalchemy import (create_engine, Column, Integer, String, Float, Text,
                        BigInteger, DateTime, ForeignKey, Boolean)
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import config


# Database Configureation Constants
HOST = config.DATABASE_HOST
USERNAME = config.DATABASE_USERNAME
PASSWORD = config.DATABASE_PASSWORD
DBNAME = config.DATABASE_NAME
PORT = config.DATABASE_PORT

# Configuring DB Connection
connection_sting = "postgresql://{}:{}@{}:{}/{}".format(USERNAME,PASSWORD,HOST,PORT,DBNAME)
engine = create_engine(connection_sting)
#Base = declarative_base()

# connect to database, merge comments with label

In [21]:
import psycopg2
# connect:
con = psycopg2.connect(connection_sting)

# query:
sql_query = """
SELECT comment_id, instagram_id, comment_text FROM comment;
"""
data = pd.read_sql_query(sql_query,con)
print(data.shape)
data.head(10)

(214927, 3)


,comment_id,instagram_id,comment_text
0,17873812639069135,463237787,Stunning make up but too much edit so idk is t...
1,17855678878168108,5435854882,This whole look is simply gorgeous! I love it!
2,17856076333162363,5433725959,👍👍👍👍👍
3,17880002698005738,697355306,@aimeemoe71
4,17881027429029492,4711266599,3D flower paper ✂️🌸🌸👌🏼
5,17881416658044611,3142239104,Absolutely stunning and great job ❤️❤️
6,17880243055039493,293323974,So flawless😍🔥
7,17880183850015941,8778809,Yay! I never see blonde brow wiz! Happy to see...
8,17874162292069219,446063965,Perfect
9,17856768136156923,5397261157,Goals


In [40]:
label = pd.read_csv("../Data/cp_comments_10k_labeled.csv")
label['comment_id'] = label['comment_id'].astype(int)
data_labeled = data.merge(label, left_on='comment_id', right_on='comment_id', how='right')
print(data_labeled.shape)
print(data_labeled.tail(100))

data_labeled = data_labeled[np.isfinite(data_labeled['instagram_id'])]
print(data_labeled.shape)
data_labeled

(10000, 4)
             comment_id  instagram_id comment_text  label
9900  17880933709024424           NaN          NaN     -1
9901  17879675281037492           NaN          NaN     -1
9902  17855821108166700           NaN          NaN     -1
9903  17856920056144208           NaN          NaN     -1
9904  17866202392082872           NaN          NaN     -1
9905  17855573488191352           NaN          NaN     -1
9906  17879654482027688           NaN          NaN     -1
9907  17855657479178900           NaN          NaN     -1
9908  17880315796063876           NaN          NaN     -1
9909  17881705423021092           NaN          NaN     -1
9910  17866280776092392           NaN          NaN     -1
9911  17856574600155300           NaN          NaN     -1
9912  17858632180141864           NaN          NaN     -1
9913  17868584920102916           NaN          NaN     -1
9914  17874096484069920           NaN          NaN     -1
9915  17858607073135956           NaN          NaN     -1
991

,comment_id,instagram_id,comment_text,label
0,17868669499113028,1.964491e+08,😍😍😍,1
1,17879944270050236,5.619575e+08,Love the make up @semaashraf,4
2,17868658465103896,4.025174e+09,👌👌👌beauuutifulll makeup!,1
3,17873858767066240,3.421440e+08,This campaign 😍💗,1
4,17881511206046638,3.957051e+09,New summer gloss in amber though😩😍😍,1
5,17867666182123338,1.241936e+09,❤️the brows @japanese.latina,1
6,17855623021169262,3.723297e+09,💞💞,1
7,17858634262142244,5.362275e+09,Wow😍,1
8,17879881855039872,3.157320e+09,😍😊😊,1
9,17874618784071540,1.647370e+09,✔️,1


In [60]:
sql_query = """
SELECT comment_id, instagram_id, comment_text FROM comment WHERE comment_id = '17858551909138600';
"""
test = pd.read_sql_query(sql_query,con)
test

,comment_id,instagram_id,comment_text


# clean, parse comments, translate emoji

In [41]:
def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return 1
    return 0

def text_has_partial_emoji(text):
    has_emoji = False
    has_text = False
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            has_emoji = True
        if character not in emoji.UNICODE_EMOJI:
            has_text = True
    return has_emoji & has_text
        
def get_text_from_partial_emoji(text):
    new_text = ''
    if(text_has_partial_emoji(text)):
        for character in text:
            if character not in emoji.UNICODE_EMOJI:
                new_text = new_text + character
    return new_text

def text_return_emoji(text):
    new_text = ''
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            new_text = new_text + character
    return new_text


def translate_emoji(text):
    new_text = ''
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            tmp = emoji.demojize(character, ('','')).replace("_", " ").lower()
            new_text = new_text + tmp + '. '
    return new_text

def get_emoji(word_list):
    new_text = ' '.join([text_return_emoji(word) for word in word_list.split()
                       if text_has_emoji(word)])
    return new_text
    

def is_username(word):
    if re.match("@[A-Za-z0-9_.]+",word):
        return True
    return False

def get_username(word_list):
    new_text = ' '.join([word.lower() for word in word_list.split()
                       if is_username(word)])
    return new_text
    
def is_hashtag(word):
    if re.match("#[A-Za-z0-9_.]+",word):
        return True
    return False

def get_hashtag(word_list):
    new_text = ' '.join([word.lower() for word in word_list.split()
                       if is_hashtag(word)])
    return new_text


def get_text(word_list):
    new_text = ''
    for word in word_list.split():
        if not is_username(word) and not is_hashtag(word) and not text_has_emoji(word):
            new_text = new_text + ' ' + word
        if text_has_partial_emoji(word):
            new_text = new_text + ' ' + get_text_from_partial_emoji(word)
    return new_text


In [45]:
parsed_comments = pd.DataFrame(columns = ['original_comment','text','emoji','translated_emoji','username','hashtag','label','purchase_int'])
parsed_comments['original_comment'] = data_labeled['comment_text']
parsed_comments['label'] = data_labeled['label']
parsed_comments['emoji'] = ''
parsed_comments['translated_emoji'] = ''

In [46]:
for index, row in parsed_comments.iterrows():
    parsed_comments.set_value(index,'username',get_username(row['original_comment']))
    parsed_comments.set_value(index,'hashtag',get_hashtag(row['original_comment']))
    parsed_comments.set_value(index,'emoji',get_emoji(row['original_comment']))
    parsed_comments.set_value(index,'text',get_text(row['original_comment']))

for index, row in parsed_comments.iterrows():    
    #parsed_comments.set_value(index,'language',detect_language(row['text']))
    parsed_comments.set_value(index,'translated_emoji',translate_emoji(row['emoji']))
    if parsed_comments['label'][index] == 4:
        parsed_comments.set_value(index,'purchase_int',int(1))
    else:
        parsed_comments.set_value(index,'purchase_int',int(0))

# sentiment analysis w/ emoji inherent translation

In [47]:
sid = SentimentIntensityAnalyzer()
for index, row in parsed_comments.iterrows():
    ss_text = sid.polarity_scores(row['text'])
    ss_translated = sid.polarity_scores(row['text'] + row['translated_emoji'])
    ss_original = sid.polarity_scores(row['original_comment'])
    parsed_comments.set_value(index,'score_text_compound',ss_text['compound'])
    parsed_comments.set_value(index,'score_text_pos',ss_text['pos'])
    parsed_comments.set_value(index,'score_text_neu',ss_text['neu'])
    parsed_comments.set_value(index,'score_text_neg',ss_text['neg'])
    parsed_comments.set_value(index,'score_trans_compound',ss_translated['compound'])
    parsed_comments.set_value(index,'score_trans_pos',ss_translated['pos'])
    parsed_comments.set_value(index,'score_trans_neu',ss_translated['neu'])
    parsed_comments.set_value(index,'score_trans_neg',ss_translated['neg'])
    #parsed_comments.set_value(index,'score_orignal_compound',ss_original['compound'])
    #parsed_comments.set_value(index,'score_orignal_pos',ss_original['pos'])
    #parsed_comments.set_value(index,'score_orignal_neu',ss_original['neu'])
    #parsed_comments.set_value(index,'score_orignal_neg',ss_original['neg'])    
parsed_comments

,original_comment,text,emoji,translated_emoji,username,hashtag,label,purchase_int,score_text_compound,score_text_pos,score_text_neu,score_text_neg,score_trans_compound,score_trans_pos,score_trans_neu,score_trans_neg
0,😍😍😍,,😍😍😍,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.8402,0.500,0.500,0.000
1,Love the make up @semaashraf,Love the make up,,,@semaashraf,,4,1,0.6369,0.583,0.417,0.000,0.6369,0.583,0.417,0.000
2,👌👌👌beauuutifulll makeup!,beauuutifulll makeup!,👌👌👌,ok hand. ok hand. ok hand.,,,1,0,0.0000,0.000,1.000,0.000,0.9230,0.877,0.123,0.000
3,This campaign 😍💗,This campaign,😍💗,smiling face with heart-eyes. growing heart.,,,1,0,0.0000,0.000,1.000,0.000,0.1779,0.221,0.779,0.000
4,New summer gloss in amber though😩😍😍,New summer gloss in amber though,😩😍😍,weary face. smiling face with heart-eyes. smil...,,,1,0,0.0000,0.000,1.000,0.000,0.7184,0.316,0.684,0.000
5,❤️the brows @japanese.latina,️the brows,❤,red heart.,@japanese.latina,,1,0,0.0000,0.000,1.000,0.000,0.0000,0.000,1.000,0.000
6,💞💞,,💞💞,revolving hearts. revolving hearts.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000
7,Wow😍,Wow,😍,smiling face with heart-eyes.,,,1,0,0.5859,1.000,0.000,0.000,0.0000,0.000,1.000,0.000
8,😍😊😊,,😍😊😊,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.9325,0.625,0.375,0.000
9,✔️,️,✔,heavy check mark.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000


# LR 1/0 w/ score_trans all

In [86]:
X = parsed_comments.as_matrix(['score_trans_compound','score_trans_pos','score_trans_neu','score_trans_neg'])
Y = parsed_comments['purchase_int'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [87]:
#accuracy on the trainin data
print(model2.score(X_train, y_train))
print(np.transpose(model2.coef_))

0.831799591002
[[ 1.44888484]
 [ 0.94873083]
 [ 1.93376127]
 [ 4.17781332]]


In [88]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print(probs)
# predict class labels for the test set
predicted = model2.predict(X_test)
print(predicted)

[[ 0.70403377  0.29596623]
 [ 0.85269779  0.14730221]
 [ 0.85269779  0.14730221]
 ..., 
 [ 0.85269779  0.14730221]
 [ 0.85269779  0.14730221]
 [ 0.81265985  0.18734015]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

In [89]:
# generate evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.799795709908
0.655631868132


In [90]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, Y, scoring='accuracy', cv=20)
print(scores)
print(scores.mean())

[ 0.8244898   0.8244898   0.8244898   0.8244898   0.8244898   0.8244898
  0.8244898   0.8244898   0.8244898   0.8244898   0.80816327  0.82786885
  0.82786885  0.82786885  0.82786885  0.82786885  0.82786885  0.82786885
  0.82786885  0.82786885]
0.825194044831


# kNN multiclass (7) w/ score_trans all

In [95]:
n_neighbors = 12
weights = 'distance'
X = parsed_comments.as_matrix(['score_trans_compound','score_trans_pos','score_trans_neu','score_trans_neg'])
Y = parsed_comments['purchase_int'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=12, p=2,
           weights='distance')

In [96]:
predicted = clf.predict(X_test)
print(predicted)
print(metrics.accuracy_score(y_test, predicted))

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [107]:
n_neighbors = 30
weights = 'uniform'
X = parsed_comments.as_matrix(['score_trans_compound','score_trans_pos','score_trans_neu','score_trans_neg'])
Y = parsed_comments['label'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')

In [108]:
predicted = clf.predict(X_test)
print(predicted)
print(metrics.accuracy_score(y_test, predicted))

[ 4  1  1  1  1  1  2  1  1  2  4  1  4  1  1  4  1  2  1  1  2  2  2  1  1
  2  4  2  1  1  2  1  2  1  1  1  1  1  1  1  1  1  1  2  1  1  4  2  2  1
  1  2  1  1  2  1  1  1  1  1  1  1  1  1  1  2  1  1  1  1  2  2  1  1  1
  1  1  1  1  4  1  1  1  4  1  1  4  4  2  1  2  1  1  1  1  1  2  1  1  1
  1  1  4  2  1  1  1 -1  1  1  1  1  4  4  1  1  1  1  1  1  1  1  2  1  1
  1  1  1  4  1  4  2  1  1  1  1  1  1  2  1  1  1  2  1  1  1  1  1  1  1
  1  4  1  1  1  4  1  1  1  1  1  1  1  2  1  1  2  1  1  1  1  1  1  2  1
  1  4  1  1  2  2  1  1  1  2  1  4  1  2  1  2  4  4  1  1  1  4  2  2  1
  4  1  1  4  1  1  4  2  1  1  2  2  4  4  1  1  1  2  1  1  1  4  1  1  4
  4  1  1  1  1  1  4  1  1  2  1  1  1  1  2  4  1  1  2  2  2  1  1  2  1
  1  1  1  1  2  1  1  1  1  1  1  1  1  1  1  1  4  1  1  1  1  2  4  4  1
  1  1  4  1  2  1  1  1  4  1  1  2  1  1  1  2  1  2  1  1  2  1  1  1  1
  2  1  4  1  1  2  1  2  4  1  4  1  1  1  2  1  1  4  4  1  2  1  1  1  1
  1  1  1  1

# sentiment analysis with emoji sentiments

In [109]:
url = 'http://kt.ijs.si/data/Emoji_sentiment_ranking/'
dfs = pd.read_html(url)
df = dfs[0]

In [110]:
EMOJI_SENTI = pd.DataFrame(columns = ['emoji','translated_emoji','unicode','compound','pos','neut','neg'])
EMOJI_SENTI['emoji'] = df['Char']
EMOJI_SENTI['translated_emoji'] = df['Unicode name']
EMOJI_SENTI['unicode'] = df['Unicodecodepoint']
EMOJI_SENTI['compound'] = df['Sentiment score[-1...+1]']
EMOJI_SENTI['pos'] = df['Pos[0...1]']
EMOJI_SENTI['neut'] = df['Neut[0...1]']
EMOJI_SENTI['neg'] = df['Neg[0...1]']
EMOJI_SENTI

,emoji,translated_emoji,unicode,compound,pos,neut,neg
0,😂,FACE WITH TEARS OF JOY,0x1f602,0.221,0.468,0.285,0.247
1,❤,HEAVY BLACK HEART,0x2764,0.746,0.790,0.166,0.044
2,♥,BLACK HEART SUIT,0x2665,0.657,0.693,0.272,0.035
3,😍,SMILING FACE WITH HEART-SHAPED EYES,0x1f60d,0.678,0.729,0.219,0.052
4,😭,LOUDLY CRYING FACE,0x1f62d,-0.093,0.343,0.220,0.436
5,😘,FACE THROWING A KISS,0x1f618,0.701,0.754,0.193,0.053
6,😊,SMILING FACE WITH SMILING EYES,0x1f60a,0.644,0.704,0.237,0.060
7,👌,OK HAND SIGN,0x1f44c,0.563,0.657,0.249,0.094
8,💕,TWO HEARTS,0x1f495,0.632,0.674,0.285,0.042
9,👏,CLAPPING HANDS SIGN,0x1f44f,0.520,0.624,0.271,0.104


In [134]:
for index, row in parsed_comments.iterrows():
    row_emoji = row['emoji']
    if row_emoji:
        temp_compound = [0.0]
        temp_pos = [0.0]
        temp_neutr = [0.0]
        temp_neg = [0.0]
        for character in row_emoji:
            if not EMOJI_SENTI[EMOJI_SENTI['emoji'].str.contains(character)].empty:
                temp_compound.append(EMOJI_SENTI[EMOJI_SENTI['emoji'].str.contains(character)]['compound'].item())
                temp_pos.append(EMOJI_SENTI[EMOJI_SENTI['emoji'].str.contains(character)]['pos'].item())
                temp_neutr.append(EMOJI_SENTI[EMOJI_SENTI['emoji'].str.contains(character)]['neut'].item())
                temp_neg.append(EMOJI_SENTI[EMOJI_SENTI['emoji'].str.contains(character)]['neg'].item())
        parsed_comments.set_value(index,'score_sentitrans_compound',np.mean(temp_compound))
        parsed_comments.set_value(index,'score_sentitrans_pos',np.mean(temp_pos))
        parsed_comments.set_value(index,'score_sentitrans_neut',np.mean(temp_neutr))
        parsed_comments.set_value(index,'score_sentitrans_neg',np.mean(temp_neg))
parsed_comments.fillna(value = 0.0)
parsed_comments

,original_comment,text,emoji,translated_emoji,username,hashtag,label,purchase_int,score_text_compound,score_text_pos,score_text_neu,score_text_neg,score_trans_compound,score_trans_pos,score_trans_neu,score_trans_neg,score_sentitrans_compound,score_sentitrans_pos,score_sentitrans_neut,score_sentitrans_neg
0,😍😍😍,,😍😍😍,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.8402,0.500,0.500,0.000,0.508500,0.546750,0.164250,0.039000
1,Love the make up @semaashraf,Love the make up,,,@semaashraf,,4,1,0.6369,0.583,0.417,0.000,0.6369,0.583,0.417,0.000,NaN,NaN,NaN,NaN
2,👌👌👌beauuutifulll makeup!,beauuutifulll makeup!,👌👌👌,ok hand. ok hand. ok hand.,,,1,0,0.0000,0.000,1.000,0.000,0.9230,0.877,0.123,0.000,0.422250,0.492750,0.186750,0.070500
3,This campaign 😍💗,This campaign,😍💗,smiling face with heart-eyes. growing heart.,,,1,0,0.0000,0.000,1.000,0.000,0.1779,0.221,0.779,0.000,0.445000,0.479000,0.153333,0.034333
4,New summer gloss in amber though😩😍😍,New summer gloss in amber though,😩😍😍,weary face. smiling face with heart-eyes. smil...,,,1,0,0.0000,0.000,1.000,0.000,0.7184,0.316,0.684,0.000,0.247000,0.420250,0.156000,0.173750
5,❤️the brows @japanese.latina,️the brows,❤,red heart.,@japanese.latina,,1,0,0.0000,0.000,1.000,0.000,0.0000,0.000,1.000,0.000,0.373000,0.395000,0.083000,0.022000
6,💞💞,,💞💞,revolving hearts. revolving hearts.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.492667,0.520000,0.120000,0.027333
7,Wow😍,Wow,😍,smiling face with heart-eyes.,,,1,0,0.5859,1.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.339000,0.364500,0.109500,0.026000
8,😍😊😊,,😍😊😊,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.9325,0.625,0.375,0.000,0.491500,0.534250,0.173250,0.043000
9,✔️,️,✔,heavy check mark.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.135000,0.198500,0.238000,0.063500


In [135]:
parsed_comments.fillna(value = 0.0)

,original_comment,text,emoji,translated_emoji,username,hashtag,label,purchase_int,score_text_compound,score_text_pos,score_text_neu,score_text_neg,score_trans_compound,score_trans_pos,score_trans_neu,score_trans_neg,score_sentitrans_compound,score_sentitrans_pos,score_sentitrans_neut,score_sentitrans_neg
0,😍😍😍,,😍😍😍,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.8402,0.500,0.500,0.000,0.508500,0.546750,0.164250,0.039000
1,Love the make up @semaashraf,Love the make up,,,@semaashraf,,4,1,0.6369,0.583,0.417,0.000,0.6369,0.583,0.417,0.000,0.000000,0.000000,0.000000,0.000000
2,👌👌👌beauuutifulll makeup!,beauuutifulll makeup!,👌👌👌,ok hand. ok hand. ok hand.,,,1,0,0.0000,0.000,1.000,0.000,0.9230,0.877,0.123,0.000,0.422250,0.492750,0.186750,0.070500
3,This campaign 😍💗,This campaign,😍💗,smiling face with heart-eyes. growing heart.,,,1,0,0.0000,0.000,1.000,0.000,0.1779,0.221,0.779,0.000,0.445000,0.479000,0.153333,0.034333
4,New summer gloss in amber though😩😍😍,New summer gloss in amber though,😩😍😍,weary face. smiling face with heart-eyes. smil...,,,1,0,0.0000,0.000,1.000,0.000,0.7184,0.316,0.684,0.000,0.247000,0.420250,0.156000,0.173750
5,❤️the brows @japanese.latina,️the brows,❤,red heart.,@japanese.latina,,1,0,0.0000,0.000,1.000,0.000,0.0000,0.000,1.000,0.000,0.373000,0.395000,0.083000,0.022000
6,💞💞,,💞💞,revolving hearts. revolving hearts.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.492667,0.520000,0.120000,0.027333
7,Wow😍,Wow,😍,smiling face with heart-eyes.,,,1,0,0.5859,1.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.339000,0.364500,0.109500,0.026000
8,😍😊😊,,😍😊😊,smiling face with heart-eyes. smiling face wit...,,,1,0,0.0000,0.000,0.000,0.000,0.9325,0.625,0.375,0.000,0.491500,0.534250,0.173250,0.043000
9,✔️,️,✔,heavy check mark.,,,1,0,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.135000,0.198500,0.238000,0.063500


# LR 0/1w/ senti score from (text + emoji_senti)/2

In [136]:
X = parsed_comments.as_matrix(['score_sentitrans_compound','score_sentitrans_pos','score_sentitrans_neu','score_sentitrans_neg'])
Y = parsed_comments['purchase_int'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [137]:
#accuracy on the trainin data
print(model2.score(X_train, y_train))
print(np.transpose(model2.coef_))

0.831799591002
[[ 1.44888484]
 [ 0.94873083]
 [ 1.93376127]
 [ 4.17781332]]


In [138]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print(probs)
# predict class labels for the test set
predicted = model2.predict(X_test)
print(predicted)

[[ 0.70403377  0.29596623]
 [ 0.85269779  0.14730221]
 [ 0.85269779  0.14730221]
 ..., 
 [ 0.85269779  0.14730221]
 [ 0.85269779  0.14730221]
 [ 0.81265985  0.18734015]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

In [139]:
# generate evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.799795709908
0.655631868132


In [140]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, Y, scoring='accuracy', cv=20)
print(scores)
print(scores.mean())

[ 0.8244898   0.8244898   0.8244898   0.8244898   0.8244898   0.8244898
  0.8244898   0.8244898   0.8244898   0.8244898   0.80816327  0.82786885
  0.82786885  0.82786885  0.82786885  0.82786885  0.82786885  0.82786885
  0.82786885  0.82786885]
0.825194044831


# kNN 0/1 w/ senti score from (text + emoji_senti)/2

In [141]:
n_neighbors = 12
weights = 'distance'
X = parsed_comments.as_matrix(['score_trans_compound','score_trans_pos','score_trans_neu','score_trans_neg'])
Y = parsed_comments['purchase_int'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=12, p=2,
           weights='distance')

In [142]:
predicted = clf.predict(X_test)
print(predicted)
print(metrics.accuracy_score(y_test, predicted))

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

# kNN multiclass w/ senti score from (text + emoji_senti)/2

In [144]:
n_neighbors = 30
weights = 'uniform'
X = parsed_comments.as_matrix(['score_trans_compound','score_trans_pos','score_trans_neu','score_trans_neg'])
Y = parsed_comments['label'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')

In [145]:
predicted = clf.predict(X_test)
print(predicted)
print(metrics.accuracy_score(y_test, predicted))

[ 4  1  1  1  1  1  2  1  1  2  4  1  4  1  1  4  1  2  1  1  2  2  2  1  1
  2  4  2  1  1  2  1  2  1  1  1  1  1  1  1  1  1  1  2  1  1  4  2  2  1
  1  2  1  1  2  1  1  1  1  1  1  1  1  1  1  2  1  1  1  1  2  2  1  1  1
  1  1  1  1  4  1  1  1  4  1  1  4  4  2  1  2  1  1  1  1  1  2  1  1  1
  1  1  4  2  1  1  1 -1  1  1  1  1  4  4  1  1  1  1  1  1  1  1  2  1  1
  1  1  1  4  1  4  2  1  1  1  1  1  1  2  1  1  1  2  1  1  1  1  1  1  1
  1  4  1  1  1  4  1  1  1  1  1  1  1  2  1  1  2  1  1  1  1  1  1  2  1
  1  4  1  1  2  2  1  1  1  2  1  4  1  2  1  2  4  4  1  1  1  4  2  2  1
  4  1  1  4  1  1  4  2  1  1  2  2  4  4  1  1  1  2  1  1  1  4  1  1  4
  4  1  1  1  1  1  4  1  1  2  1  1  1  1  2  4  1  1  2  2  2  1  1  2  1
  1  1  1  1  2  1  1  1  1  1  1  1  1  1  1  1  4  1  1  1  1  2  4  4  1
  1  1  4  1  2  1  1  1  4  1  1  2  1  1  1  2  1  2  1  1  2  1  1  1  1
  2  1  4  1  1  2  1  2  4  1  4  1  1  1  2  1  1  4  4  1  2  1  1  1  1
  1  1  1  1